# Getting data collection up 
### [RN] 2/13/2021 11:34

I have a scraper up and working. This is a bash script. 

I will let the data collection (still workign fine) run for one hour on the hour. The data scraper will add the weather conditions to the data file. I will also automate processing the raw data into hits per minute so that a histogram of each hour may be auto-generated. I will do this using bash scripts for control/scraper and python for the cosmic ray data. 


In [ ]:
#!/bin/bash
  
# scrape weather data from first line in table of url = https://forecast.weather.gov/data/obhistory/metric/KSBA.html

URL="https://forecast.weather.gov/data/obhistory/metric/KSBA.html"
RAW=$(curl $URL )
CUT=$(echo $RAW | grep -oP '(?<=</table></form>).*?(?=</th></tr></table>)' | grep -oP '(?<=</th><th).*?(?=Time<br>)')
echo $CUT
echo 
TIME=$(echo $CUT | grep -oP '(?<=align="right">).*?(?=</td><td>)' | cut -d' ' -f1)
NOW=$(echo $TIME | cut -d' ' -f1)
echo "time"
echo $NOW

WC=$(echo $CUT | grep -oP '(?<=align="left">).*?(?=<td>)' | cut -d'>' -f1)
echo $WC
WN=$(echo $WC | cut -d'<' -f1)
echo "weather conditions"
echo $WN

TABLE=$(echo $RAW| grep -oP '(?<=/th>).*?(?=</th>)' )
echo "table"
echo $TABLE 

LINE=$(echo $TABLE | sed -e 's/\ //g' | grep -oP '(?<=<td>).*?(?=/td>)')
echo 
echo "line"
echo $LINE

DATE=$(echo $LINE | cut -d' ' -f1 | sed -e 's/<//g')
WIND=$(echo $LINE | cut -d' ' -f2 | sed -e 's/<//g')
VIS=$(echo $LINE | cut -d' ' -f3 | sed -e 's/<//g')
SKY_COND=$(echo $LINE | cut -d' ' -f4 | sed -e 's/<//g')
AIR_T=$(echo $LINE | cut -d' ' -f5 | sed -e 's/<//g')
DWPNT=$(echo $LINE | cut -d' ' -f6 | sed -e 's/<//g')
MAX6HR=$(echo $LINE | cut -d' ' -f7 | sed -e 's/<//g')
MIN6HR=$(echo $LINE | cut -d' ' -f8 | sed -e 's/<//g')
HUMID=$(echo $LINE | cut -d' ' -f9 | sed -e 's/<//g')
WIND_CHILL=$(echo $LINE | cut -d' ' -f10 | sed -e 's/<//g')
HEAT_IND=$(echo $LINE | cut -d' ' -f11 | sed -e 's/<//g')
ALT=$(echo $LINE | cut -d' ' -f12 | sed -e 's/<//g')
SEA_LEV=$(echo $LINE | cut -d' ' -f13 | sed -e 's/<//g')
PCPT_1HR=$(echo $LINE | cut -d' ' -f14 | sed -e 's/<//g')
PCPT_3HR=$(echo $LINE | cut -d' ' -f15 | sed -e 's/<//g')
PCPT_6HR=$(echo $LINE | cut -d' ' -f16 | sed -e 's/<//g')

OUT="$DATE, $NOW, $WIND, $VIS, $WN, $SKY_COND, $AIR_T, $DWPNT, $MAX6HR, $MIN6HR, $HUMID, $WIND_CHILL, $HEAT_IND, $ALT, $SEA_LEV, $PCPT_1HR, $PCPT_3HR, $PCPT_6HR"

"WeatherScrape.sh" 49L, 1889C                                                                                                                                                           1,1           Top


Now i am working on a python script to collect cosmic ray data for exactly an hour and add the weather data. I think i will use multiprocessing to make sure timing is precise. 

I am setting a daemon thread to do timeout, a shell script to make the header of the raw data file, multiprocessing to deal with data collection 

Here is my script that gets data from the detector. I will set it to run about a minute before every hour, and wait to collect data until on the top of the hour. After the hour is up all the data processing and uploading will happen. This should allow for pretty continous data collection while still binning data by the hour. 

Data processing will be finding events per minutes for each hour. 

I would also like to have a total events per minute average across an hour and a file of all weather data points.


In [ ]:
### 1hr_CosmicRayCollect.py

#!/usr/bin/env python3

import numpy  as  np
import sys
import os
import subprocess
from threading import Thread
from multiprocessing import Process, Pipe
import _thread
import RPi.GPIO as GPIO
import time
import datetime
import sys

def CheckDir(f, ff):
    i = 2               # format for f should be : NAME_n.txt with n = 1, 2, 3.....
    while True:
        try:
            ls = subprocess.check_output(['ls ' + f + " >/dev/null 2>&1"], shell = True)
            f = eval(ff)
            i = i + 1
        except:
            break
    out = str(f)
    exp = str(i - 1)
    return out, exp

def TimeOut(st, duration, RawDat, child_conn):
    print(time.perf_counter() - st)
    time.sleep(duration - (time.perf_counter() - st))   # set timeout for operations defined in this class
    print("ending program", time.perf_counter() - st)
    done=True
    child_conn.close()
    RawDat.close() # close df

def CollectData(st, child_conn, RawDat):
    count = 0
    to = Thread(target = TimeOut, args=(st, 1800, RawDat, child_conn,)) # i want to collect data in precise time window
    to.setDaemon(True)
    to.start() # set timeout 
    while done==False:
        try:
            if GPIO.input(37)==1:
                count = count + 1
                currentTime = datetime.datetime.now()
                line = str(count) + ", " + str(time.perf_counter() - st) + ", " + str(currentTime) + "\n"
                RawDat.write(line)
                while GPIO.input(37)==1:
                    time.sleep(0.00001)
        except: # timeout kicked in during collection 
            print("Last attempt to send data: t = " + str(time.perf_counter() - st))
            to.join()
            os._exit(0)

# Setup Raw Data file
st = time.perf_counter()
RawDat, HourN = CheckDir("Hour_1", '"Hour_" + str(i)')
subprocess.Popen(["mkdir " + RawDat], shell=True)
subprocess.check_output(["ls " + RawDat], shell=True)
RawDat = "./" + RawDat + "/Raw.csv"
print(RawDat)

subprocess.check_output(["./StartDAQ.sh " + RawDat], shell=True) # get most recent weather data
print("Got weather data")

# get ready for data acq
WriteRaw = open(RawDat, 'a+')

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)

global done # flag 
done=False

# Setup DAQ as child process so it can be interrupted as soon as DAQ time is up; avoid duplicate data points
parent_conn, child_conn = Pipe() # multiprocessing to avoid timing issues

# wait until exact time to start. This is so i can start the initialization ahead of time and be ready to get precise time bins.
while time.perf_counter() - st <= 5: # wait from initialization to start DAQ 
    pass
# start data collection as child process
DAQ_st = time.perf_counter()
GetData = Process(target = CollectData, args = (DAQ_st, child_conn, WriteRaw,)) # Data collection as child process

print(DAQ_st)
GetData.start()
# join when done 
GetData.join()
print(time.perf_counter() - DAQ_st)
print("proceed")

### Add weather to df header 

subprocess.check_output(["AddWeather.sh " + RawDat + " > /dev/null"], shell = True)



In [ ]:
### StartDAQ.sh

#!/bin/bash
  
echo $1
DAY=$(date +%D)
TIME=$(date +%T)
echo "$DAY $TIME" > $1
echo "Day of month, Time of data collection, Wind speed (kph), Visibility (km), Weather, Sky COnditions, Air Temp (C), Dewpoint (C), 6 Hour Max Temp
, 6 Hour Min Temp, Relative Humidity (%), Wind Chill (C), Heat Index (C), Alitimeter Pressure (cm), Sea Level (mb), Precipitation 1h r (cm), Precipi
tation 3hr (cm), Precipitation 6h (cm)" >> $1
echo "INSERT" >> $1
echo "---" >> $1
echo "Incident number, relative time since start, timestamp" >> $1
~                                                                                                                                                   
~                                                                                                                                                   
~                                                                      

In [ ]:
### AddWeather.sh

!/bin/bash
  
# scrape weather data from first line in table of url = https://forecast.weather.gov/data/obhistory/metric/KSBA.html

URL="https://forecast.weather.gov/data/obhistory/metric/KSBA.html"
RAW=$(curl $URL )
CUT=$(echo $RAW | grep -oP '(?<=</table></form>).*?(?=</th></tr></table>)' | grep -oP '(?<=</th><th).*?(?=Time<br>)')
echo $CUT
echo 
TIME=$(echo $CUT | grep -oP '(?<=align="right">).*?(?=</td><td>)' | cut -d' ' -f1)
NOW=$(echo $TIME | cut -d' ' -f1)
echo "time"
echo $NOW

WC=$(echo $CUT | grep -oP '(?<=align="left">).*?(?=<td>)' | cut -d'>' -f1)
echo $WC
WN=$(echo $WC | cut -d'<' -f1)
echo "weather conditions"
echo $WN

TABLE=$(echo $RAW| grep -oP '(?<=/th>).*?(?=</th>)' )
echo "table"
echo $TABLE 

LINE=$(echo $TABLE | sed -e 's/\ //g' | grep -oP '(?<=<td>).*?(?=/td>)')
echo 
echo "line"
echo $LINE

DATE=$(echo $LINE | cut -d' ' -f1 | sed -e 's/<//g')
WIND=$(echo $LINE | cut -d' ' -f2 | sed -e 's/<//g')
VIS=$(echo $LINE | cut -d' ' -f3 | sed -e 's/<//g')
SKY_COND=$(echo $LINE | cut -d' ' -f4 | sed -e 's/<//g')
AIR_T=$(echo $LINE | cut -d' ' -f5 | sed -e 's/<//g')
DWPNT=$(echo $LINE | cut -d' ' -f6 | sed -e 's/<//g')
MAX6HR=$(echo $LINE | cut -d' ' -f7 | sed -e 's/<//g')
MIN6HR=$(echo $LINE | cut -d' ' -f8 | sed -e 's/<//g')
HUMID=$(echo $LINE | cut -d' ' -f9 | sed -e 's/<//g')
WIND_CHILL=$(echo $LINE | cut -d' ' -f10 | sed -e 's/<//g')
HEAT_IND=$(echo $LINE | cut -d' ' -f11 | sed -e 's/<//g')
ALT=$(echo $LINE | cut -d' ' -f12 | sed -e 's/<//g')
SEA_LEV=$(echo $LINE | cut -d' ' -f13 | sed -e 's/<//g')
PCPT_1HR=$(echo $LINE | cut -d' ' -f14 | sed -e 's/<//g')
PCPT_3HR=$(echo $LINE | cut -d' ' -f15 | sed -e 's/<//g')
PCPT_6HR=$(echo $LINE | cut -d' ' -f16 | sed -e 's/<//g')

OUT="$DATE, $NOW, $WIND, $VIS, $WN, $SKY_COND, $AIR_T, $DWPNT, $MAX6HR, $MIN6HR, $HUMID, $WIND_CHILL, $HEAT_IND, $ALT, $SEA_LEV, $PCPT_1HR, $PCPT_3HR, $PCPT_6HR"

sed -i "s/INSERT/$OUT/g" $1
~                                             

### [RN] 6:20 pm 

took a pretty long break. now working on data processng. 

I did a 30 minute test and these scripts run well.

I updated the run script so that there will be a summary of the data from each hour, "TotalOverTime.csv" for conveninece. This gives the average hits per minute over the hour, standard deviation, variange, and range in event per minute numbers, as well as the environmental conditions for that hour. 


Updated 1hr run below. 


In [ ]:
### 1 hr run script, keeping track of totals as well. (1hr_CosmicRayCollect.py)

#!/usr/bin/env python3

import scipy.stats as stats
import numpy  as  np
import sys
import os
import subprocess
from threading import Thread
from multiprocessing import Process, Pipe
import _thread
import RPi.GPIO as GPIO
import time
import datetime
import sys

path = "/home/pi/Desktop/128_CosmicRays/"
def CheckDir(f, ff):
    i = 2               # format for f should be : NAME_n.txt with n = 1, 2, 3.....
    while True:
        try:
            ls = subprocess.check_output(['ls ' + path + "/" + f + " >/dev/null 2>&1"], shell = True)
            f = eval(ff)
            i = i + 1
        except:
            break
    out = str(path + "/" + f)
    exp = str(i - 1)
    return out, exp

def TimeOut(st, duration, RawDat, child_conn):
    print(time.perf_counter() - st)
    time.sleep(duration - (time.perf_counter() - st))   # set timeout for operations defined in this class
    print("ending program", time.perf_counter() - st)
    done=True
    child_conn.close()
    RawDat.close() # close df

def CollectData(st, child_conn, RawDat):
    count = 0
    to = Thread(target = TimeOut, args=(st, 3600, RawDat, child_conn,)) # i want to collect data in precise time window
    to.setDaemon(True)
    to.start() # set timeout 
    while done==False:
        try:
            if GPIO.input(37)==1:
                count = count + 1
                currentTime = datetime.datetime.now()
                line = str(count) + ", " + str(time.perf_counter() - st) + ", " + str(currentTime) + "\n"
                RawDat.write(line)
                while GPIO.input(37)==1:
                    time.sleep(0.00001)
        except: # timeout kicked in during collection 
            
# Setup Raw Data file
st = time.perf_counter()
OutDir, HourN = CheckDir("Hour_1", '"Hour_" + str(i)')
subprocess.Popen(["mkdir " + OutDir], shell=True)
subprocess.check_output(["ls " + OutDir], shell=True)
RawDat = OutDir + "/Raw.csv"
print(RawDat)

subprocess.check_output([path + "/StartDAQ.sh " + RawDat], shell=True) # get most recent weather data
print("Got weather data")

# get ready for data acq
WriteRaw = open(RawDat, 'a+')

GPIO.setmode(GPIO.BOARD)
GPIO.setup(37,GPIO.IN)

global done # flag 
done=False

# Setup DAQ as child process so it can be interrupted as soon as DAQ time is up; avoid duplicate data points
parent_conn, child_conn = Pipe() # multiprocessing to avoid timing issues

# wait until exact time to start. This is so i can start the initialization ahead of time and be ready to get precise time bins.
while time.perf_counter() - st <= 60: # wait from initialization to start DAQ 
    pass
# start data collection as child process
DAQ_st = time.perf_counter()
GetData = Process(target = CollectData, args = (DAQ_st, child_conn, WriteRaw,)) # Data collection as child process

print(DAQ_st)
GetData.start()
# join when done 
GetData.join()
print(time.perf_counter() - DAQ_st)
print("proceed")

### Add weather to df header 

subprocess.check_output([path + "/AddWeather.sh " + RawDat + " > /dev/null"], shell = True)
### Process data


def ReadFile(infile):
    read = open(infile, 'r')
    lines = read.readlines()
    read.close()
    return lines

raw = ReadFile(RawDat)
print(raw)
event_rt = []
for i in range(7, len(raw)):
    temp = raw[i].split(', ')
    print(temp)
    event_rt.append(float(temp[1])/60)


EPM = [] # events per minute
minutes = 0
count = 0
print(event_rt)
for i in range(len(event_rt)):
    if event_rt[i] - minutes < 1:
        count = count + 1
    else:
        minutes = minutes + 1
        EPM.append(count)
        count = 1
EPM.append(count) # last unfill bin 

# make header for new df for processed data 
ProcDat = OutDir + "/EventsPerMin.csv"
subprocess.Popen(["head -4 " + RawDat + " > " + ProcDat], shell=True)
subprocess.check_output(["ls > /dev/null"], shell = True)

#write out events per minute
WriteEPM = open(ProcDat, 'a+')
WriteEPM.write("Minute since start, Events per minute\n")

for i in range(len(EPM)):
    print(EPM[i])
    WriteEPM.write(str(i+1) + ", " + str(EPM[i]) + "\n")
WriteEPM.close()

# Add to total file

EnvCond = raw[2].strip()

avg = stats.tmean(EPM)
std = stats.tstd(EPM)
var = stats.tvar(EPM)
rng = max(EPM) - min(EPM)

TotalDat = path + "/TotalOverTime.csv"
WriteTotal = open(TotalDat, 'a+')
DataSummary = str(avg) + ", " + str(std) + ", " + str(var) + ", " + str(rng)
WriteTotal.write("- HOUR" +  str(HourN) + " -DATA: " + DataSummary + " -CONDITIONS: " + EnvCond + "\n")
WriteTotal.close()


Everything is working nicely now. I am having a hard time automating data upload to git though. I am tired so i will just leave this running for a while to see if there are any bugs I cannot see short term. 

To have this run, i am setting 1hr_CosmicRayCollect.py to run at every 59th minute on the hour. 1hr_CosmicRayCollect.py will get configure output files, then wait to start collecting data until it has been 60 seconds since initialization, which should be the top of the hour. It will leave an INSERT flag at configuration. Incident coscmic rays will be detected and recorded by time in count in the "Raw.txt" file in the new directory created for that hour. After exactly one hour, data collection will stop and the weather will be read out using the scraper and added to the data files. Total data sheet will also be updated. Crontab will keep these scripts running over time for continous data collection despite processing times thanks to the multiprocessing approach. 

